---
title: Introduction to Computational Chemistry with Psi4
authors:
  - name: Author Name
    email: example@cuny.edu
    affiliations:
      - ror: 00g2xk477
      - institution: CUNY – Hunter College
      - department: Chemistry
date: 2024-01-01
numbering:
  heading_2: true
  heading_3: true
---

### Purpose

In this lab, we'll be using a library called [Psi4](https://psicode.org/) to perform some _ab initio_ calculations on atoms and small molecules. You'll see how to use Psi4 to make simple calculations of energy, geometry, and vibrational modes, as well as seeing how to display this data and compare some computational methods. 

### Library import
We import all the required Python libraries

In [ ]:
# File handling
from pathlib import Path

# Data manipulation
import numpy as np
import scipy as sp

# Visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-notebook')
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

:::{note} 
The block of code below initializes Psi4 and gets rid of any leftover files from the previous run. 
:::

In [ ]:
# Psi4 library
import psi4
for x in Path().glob('psi.*.clean'): x.unlink() 

psi4.core.be_quiet()

## Problem I

Below is the general setup for a Psi4 calculation.

In [ ]:
# Sample HF/STO-3G H atom Computation

# Begin by defining the geometry of the system. 
h_atom = psi4.geometry('H')

basis = 'sto-3g'
psi4.set_options({
                 'reference': 'uhf',
                 })

In the next cell, we tell Psi4 to calculate the energy of the system and save the energy and wavefunction to variables. We then extract the number of basis functions (`nbf`) from the wavefunction data. 

In [ ]:
# Input the code from the manual below and execute it. 
# It should print out the energy and number of basis functions. 

### Basis set comparison

Below, we'll set up our loop to calculate energies with a variety of basis functions. You can set the value for `basis` as your loop variable and iterate the loop over a list of strings (the basis function names). 

In [ ]:
# Create your loop here

### Method/basis comparison

In the following cells, we calculate the energy with several methods and basis sets, saving each into lists stored in a dictionary. Each method is assigned as a dictionary key and the value is the basis set energies saved as a list. The number of basis functions are saved to a separate list called `bs_size`. The next cell shows the student how to display dictionary data in tabular format, and the following cell shows that tabular data in a Markdown cell. 

In [ ]:
# Fill in the missing information for `methods` and `bases`

methods = 
bases = 
hydrogen_energies = dict()

for method in methods:
    if method not in hydrogen_energies:
        hydrogen_energies[method] = []
        bs_size = []
    for basis in bases:
        energy, wfn = psi4.energy(method + '/' + basis, return_wfn=True)
        bs_size.append(wfn.basisset().nbf())
        hydrogen_energies[method].append(energy)

hydrogen_energies

In [ ]:
# As students often look for a way to nicely print tabular data, here is a brief example. 
from tabulate import tabulate 
table = tabulate(hydrogen_energies, headers='keys', tablefmt='html')

Evaluate this cell to see the code used to render the table created above. 

{eval}`table`

In the next cell, the data from the prior dictionary is plotted using a simple loop over the dictionary keys. The horizontal line represents the analytic solution for the molecule. 

In [ ]:
for method in methods:
    plt.plot(bs_size, hydrogen_energies[method], label=method)

plt.axhline(y=-0.5, linestyle='--')
plt.legend()

## Problem II

### Hydrofluoric acid bond length

The cell below sets up almost everything for your hydrofluoric acid bond scan. You'll need to fill in values for `method` and `basis`. 

In [ ]:
# HF molecule geometry curve

method =
basis = 

# Use f-strings to fill in values for method, basis
# 'H 1 r' means connect 'H' to the first atom with a vector of length 'r'
# We will define 'r' later. 
hf_bond = psi4.geometry(f"""
F 
H 1 r
 
""")

In [ ]:
# Set the bond to 1.0 atomic unit and perform the energy calculation.
# Be sure to save the wavefunction data in addition to the energy. 

In [ ]:
# Display all the variables available from this calculation
wfn.variables()

In [ ]:
# Print out the variable data for 'CURRENT DIPOLE' 
# (or 'current dipole', case doesn't matter)


Next, you'll do a series of energy calculations called a bond scan (or potential energy surface, PES) scan. 

In [ ]:
# Set up your bond scan loop below


In [ ]:
# Plot your data, then find and print the minimum energy corresponding bond length. 

### HF atomization energy

Perform your calculations of atomization energy below. 

### HF dipole moment

If you saved the dipole moments in your loop above, go ahead and use them. If not, set up a calculation loop again and save the dipole moments from the wavefunction outputs of the energy calculation. Find the dipole moment at the equilibrium bond length. 

In [ ]:
# Plot the dipole moment against the bond length and print the 
# dipole moment at the minimum energy bond length. 

### Repeat using PBE0 for the method

Perform the requisite calculations below using different variable names. When finished, compare the resultant plots and values against the data obtained using the Hartree-Fock method previously used. 

In [ ]:
# Perform calculations below

## Problem III: Hydronium cation

In [ ]:
# We need a few extra libraries for the next step. 
import cclib # computational chemistry library to translate formats
import json # create a JSON object from the cclib data
import openchemistry as oc # helpful display utilities for molecular data

### Planar geometry calculations

In the next cell, we do several new steps to prepare for molecular visualizations. First, we need to specify a log file for the calculations, then tell Psi4 to save the results of our calculation (until now, we've been storing the results of our small calculations in temporary memory). Once we set up the log file, we'll perform a geometry optimization using `psi4.optimize()`, then calculate the vibrational frequencies in the system with `psi4.frequency()`. Finally, in order to properly parse the log file, we'll print a closing line to it and close the file. 

In [ ]:
# H3O+, planar geometry calcs

# For this calculation, we want to save the output file for later retrieval
# Set a folder name and file name.
outfolder = Path(
outfolder.mkdir(exist_ok=True) # Create a new directory, no error if it already exists
outfile = Path(outfolder, # fill in a filename
# Remove old file if it exists 
outfile.unlink(missing_ok=True)
# Make sure Psi4 prints the outfile and all required components are present
psi4.set_output_file(outfile, append=True, print_header=True)

# Now we set up the molecule
charge = 
multp = 

method =
basis = 

# Read in the contents of our geometry file
with open('inputs/geom_planar.xyz', 'r') as f:
    planar_geom = f.read()

# Again, use f-strings to fill in values for charge, multp, geometry, etc.
planar_h3o = psi4.geometry(f"""
{charge} {multp}

{planar_geom}

""")

## Set options and run the geometry optimization
# This lets us avoid calling the basis for each calculation step.
# Must set reference to 'uhf' or 'rhf' to get IR intensities.
psi4.set_options({
    'reference': 'rhf',
    'basis': basis})

psi4.optimize(method)

# Now calculate the vibrational frequencies for the system
psi4.frequency(method)

# Close the outfile with success line 
if type(psi4.variable('current energy')) is float:
    psi4.extras.exit_printing(success=True)
    psi4.core.close_outfile()

The next cell uses `cclib`, a library for parsing computational chemistry files, to convert our Psi4 file to a general use format called "Chemical JSON". We'll use this file in the following cell to create an OpenChemistry `Molecule` object which implements a number of visualization methods. 

In [ ]:
# Import and parse Psi4 data with cclib 
planar_log = cclib.io.ccread(outfile)

# Display some information about our log file, make sure the import succeeded
for key, value in planar_log.metadata.items():
    print(f'{key:18}: {value}')

In [ ]:
# Convert cclib data to cJSON output that OpenChemistry can understand
cjsonfile = outfile.with_suffix('.cjson')
# cclib doesn't yet work with Path objects, so we convert it to a string
cclib.io.ccwrite(planar_log, outputtype='cjson', outputdest=str(cjsonfile))

with open(cjsonfile) as f:
    planar_cjson = json.load(f)
    
planar_mol = oc.load(planar_cjson)

Now that we have an OpenChemistry Molecule, we'll do some visualization and retrieve some of the information about it. During the import process, cclib converts some of the calculation data to alternate units. Energies are converted to electron volts (eV). 

In [ ]:
print(f'The total energy of the molecule is {planar_mol.properties.data()['energy']['total']:0.4f} eV.')
planar_mol.structure.show()

The default representation of the molecule doesn't define any bonds. In the next cell, we will define bonds between the central oxygen atom (atom 0) and each of the hydrogen atoms. 

In [ ]:
## For visualization purposes, create bonds between oxygen and all other atoms
# Atoms are numbered consecutively from 0 in the order they're listed in the 
# input file. We want bonds from the first atom (oxygen, number "0") to each
# other atom. 
planar_mol.structure.data()['bonds'] = {'connections': {'index': [0, 1,
                                                                0, 2,
                                                                0, 3]}}
# Set all bonds to order 1 (single bonds)
planar_mol.structure.data()['bonds']['order'] = [1, 1, 1]

The next cell renames a couple of the cclib variables so the OpenChemistry visualizer recognizes the fields. Details are given in the code comments below. 

In [ ]:
## Need to edit the vibrational data that comes in from cclib
# Need to rename 'displacement' to 'eigenVectors' so that OpenChemistry can 
# recognize it. Also need to reshape the array (group by vibrational mode).
planar_mol.vibrations.data()['eigenVectors'] = \
    np.array(
        planar_mol.vibrations.data()['displacement']).reshape(
            len(planar_mol.vibrations.data()['frequencies'])
            , -1
        ).tolist()

# Change 'intensities' to contain _only_ IR data, throw out Raman information
planar_mol.vibrations.data()['intensities'] = planar_mol.vibrations.data()['intensities'].pop('IR')

# Add enumerated list of 'modes' for OC to reference
planar_mol.vibrations.data()['modes'] = list(range(0, (len(planar_mol.vibrations.data()['intensities']))))

In the next three cells, we will get to view the information about the molecule. The first cell displays a 3D representation of the molecule. You can use the mouse to rotate the molecule (click and drag), or to zoom in and out using the scroll wheel. 

The second cell shows a summary of the molecular vibrations and gives the IR intensity of each. Using the visualization in the third cell, you can click on the lines in the spectrum to see which vibration results in that spectral line. Some of the vibrations overlap, so you'll have to manually select them. Click on the vertical ellipsis ($\scriptsize\vdots$) in the upper right corner of the visualization to reveal the menu. Select each of the six normal modes to see the vibrational mode and the location of its spectral line. 

In [ ]:
# Show the molecule
planar_mol.structure.show()

In [ ]:
# Print out a table of information about the IR vibrational modes
planar_mol.vibrations.table()

In [ ]:
planar_mol.vibrations.show()

### Repeat the above steps with data from the Pyramidal structure

Using the same steps with the `geom_pyramidal.xyz` file in the Pyramidal folder, run the  calculation, import the new output file using `cclib`, convert it to a cJSON file and then import that with `oc.load()`, and fix the bonds. Then display the molecule, list the vibration table, and visualize the vibrations.

You'll want to change the name of the data from `planar_mol` to something else (I recommend something understandable, such as `pyramidal_mol`. `pyr_mol` will also work in a pinch.). Anywhere you referenced `planar_mol` in the last section needs to be changed to reference the new data. 

### Potential-Energy Surface Scan

To start, we need to measure the bond length of the O-H bond in the pyramidal structure. We will make a function to extract the atomic coordinates from the dictionary of information about a molecule (`atom_coords()`). We'll then define a function called `bond_length()` which will take a pair of coordinates as input and output a distance. Since the SciPy library already has an optimized function for this task, we will just "rebrand" it.

In [ ]:
# Define bond_length() function using the SciPy Euclidian distance function
from scipy.spatial import distance
def bond_length(atom1,atom2):
    '''Calculate distance between two objects in space'''
    dist = distance.euclidean(atom1, atom2)
    return dist


# Define a function to make a matrix of coordinates for all atoms in a molecule
def atom_coords(molecule): 
    '''Takes an openchemistry._molecule.Molecule and pulls atomic coordinates 
        out as a 3 x N array of xyz coordinates. '''
    atom_info = molecule.structure.data()['atoms']
    coord_list = atom_info['coords']['3d']
    n_atoms = atom_info['elements']['atom count']
    return np.array(coord_list).reshape(n_atoms,-1)
 

Now, insert the name of your pyramidal molecule into the `atom_coords()` function and execute the cell to print out the length of the O-H bond in this molecule. 

In [ ]:
# Take the 3N length list of atomic coordinates and split it by the number of 
# atoms in our molecule
atom_locs = atom_coords(pyramidal_mol)

# Now, we print out the result with a (more) reasonable number of significant 
# figures using the string formatting rule `:.4` to output four figures after 
# the decimal. 

OH_bond_length = bond_length(atom_locs[0],atom_locs[1])
print(f'The length of the O-H bond is {OH_bond_length:.4f} Angstrom')

The H{sub}`3`O{sup}`\+` molecule is initialized below using the $z$-matrix file discussed in the lab guide. You'll need to set the values for `charge`, `multp`, `method`, and `basis`, as well as setting the `outfile` location. 

In [ ]:
# H3O+, PES scan

# Make a new outfile for the PES scan data.
outfile = Path(outfolder, # add a file name here

# Remove old file if it exists 
outfile.unlink(missing_ok=True)
# Make sure Psi4 prints the outfile and all required components are present
psi4.set_output_file(outfile, append=True, print_header=True)

charge = 
multp = 

# Read in the contents of our geometry file
with open('inputs/h3o.zmat', 'r') as f:
    h3o_zmat = f.read()

method = 
basis = 

# Use f-strings to fill in values for charge, basis, geometry
# 'H 1 r' means connect 'H' to the first atom with a vector of length 'r'
# We will define 'r' later. 
h3o = psi4.geometry(f"""
{charge} {multp}

{h3o_zmat}
""")

h3o.a = OH_bond_length

psi4.set_options({
                 'basis': basis,
                 })

In the next cell, set up the loop to scan the H-O-X bond angle between 90° and 135°. Make sure to save the energies to a list so you can plot the values agains the angle. Also remember that Psi4 reports energies in atomic units ($E_\textup{h}$), so you'll need to make a conversion before comparing the value to the planar and pyramidal calculations. 

In [ ]:
# Write a loop to calculate the energy for each HOX angle
# in the range (90, 135) in single degree increments


In the next cell, find the minimum value in the PES scan and calculate the difference in energy between the minimum and the local maximum (the energy of the structure at 90 degrees). If you name your variables `min_energy`, `min_angle`, `transition_energy`, and `barrier_energy`, you should be able to evaluate the Markdown cell below and get those values printed within the text. Notice the unit conversion being done for the `planar_mol` energy value (eV $\rightarrow E_\text{h}$) You might need to evaluate twice. Use this trick with <code>\{eval\}\`code\`</code> to embed code outputs in your writing. You can also save plots like this and display them in your discussion. Details on this process are available on the [MyST Markdown user guide](https://mystmd.org/guide/reuse-jupyter-outputs). 

In [ ]:
# Find the minimum and transition anergies and the angle a the minimum. 
# You'll also need to calculate the barrier energy. 

The minimum energy is {eval}`f'{min_energy:1.4f}'` $E_\text{h}$, which occurs at {eval}`min_angle`°.
The energy of the transition state (at 90°) is {eval}`f'{transition_energy:1.4f}'` $E_\text{h}$.
The optimized energy of the planar ion is {eval}`np.round(planar_mol.properties.data()['energy']['total'], 5) / 27.211` $E_\text{h}$.
The barrier height is {eval}`barrier_energy` $E_\text{h}$.

## Results

Describe and comment the most important results.

## Suggested next steps

State suggested next steps, based on results obtained in this notebook.

## References

We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2